In [1]:
# Copyright (c) 2016, Vladimir Feinberg
# Licensed under the BSD 3-clause license (see LICENSE)

%matplotlib inline 
import matplotlib.pyplot as plt

import sys
import math

import contexttimer
import numpy as np
import scipy.linalg
import scipy.sparse.linalg

from runlmc.linalg.kronecker import Kronecker
from runlmc.linalg.sum_matrix import SumMatrix
from runlmc.linalg.toeplitz import Toeplitz
from runlmc.linalg.numpy_matrix import NumpyMatrix
import runlmc.util.testing_utils as utils

from scipy.fftpack import fft, ifft
from scipy.sparse.linalg import LinearOperator

def chan(t): # could be parallel...
    n = len(t)
    inc = np.arange(n)
    l = t * (n - inc)
    l = l.astype(float)
    r = t[::-1][:-1] * inc[1:]
    l[1:] += r
    return l / n

def inv_precond(lus, perm, inv_perm, x):
    D = lus[0][0].shape[0]
    x = x.astype('complex').reshape(D, -1)
    x = fft(x, overwrite_x=True).reshape(-1)
    x = x[perm].reshape(-1, D)
    x = np.hstack([scipy.linalg.lu_solve(factors, b) for factors, b in zip(lus, x)])
    x = x[inv_perm].reshape(D, -1)
    return ifft(x, overwrite_x=True).reshape(-1)

def stress_sum_solve(my_mat):
    b = np.random.rand(my_mat.shape[0])
    sz = len(b)
    np_mat = my_mat.as_numpy()
    linop = my_mat.as_linear_operator()

    tol = 1e-6 # min(np.finfo('float64').eps * sz * max(math.log(sz), 1) * 2, 1e-10)
    cond = np.linalg.cond(np_mat)
    print('    cond {} tol {:g}'.format(cond, tol))
    e = 1e-5
    
    Bs = np.array([x.A.A for x in my_mat.Ks])
    Bs = np.concatenate((Bs, [np.identity(len(Bs[0]))])) #?
    tops = np.array([x.B.top for x in my_mat.Ks])
    z = np.zeros_like(tops[0])
    z[0] += e
    tops = np.concatenate((tops, [z])) #?
    toep_blocks = np.tensordot(Bs, tops, axes=(0, 0))
    circ_pre = np.array([[chan(toep_blocks[i, j])
        for j in range(toep_blocks.shape[1])] for i in range(toep_blocks.shape[0])])                         
    circ_eigs = fft(circ_pre, overwrite_x=True) # applies to last axis
    rc = np.rollaxis(circ_eigs, -1, 0)
    lus = [scipy.linalg.lu_factor(dd, overwrite_a=True) for dd in rc]
    # the rolled axis perm
    perm=np.add.outer(np.arange(circ_eigs.shape[2]), np.arange(circ_eigs.shape[0]) * circ_eigs.shape[2]).ravel()
    inv_perm = np.zeros(len(perm), dtype=int)
    inv_perm[perm] = np.arange(len(perm))
    lx = lambda x: inv_precond(lus, perm, inv_perm, x).real
    pre = LinearOperator((sz, sz), matvec=lx, rmatvec=lx)

    def time_method(f):
        with contexttimer.Timer() as solve_time:
            solve, name = f()
        print('    {} sec {:8.4f} resid {:8.4e}'.format(
            name.rjust(20),
            solve_time.elapsed,
            np.linalg.norm(linop.matvec(solve) - b)))
    time_method(lambda: (np.linalg.solve(np_mat, b), 'linear solve'))

    def sparse():
        out, succ = scipy.sparse.linalg.cg(my_mat, b, tol=tol, maxiter=sz)
        return out, '{} sparse CG'.format('' if not succ else '*')
    time_method(sparse)
    
    def sparsep():
        out, succ = scipy.sparse.linalg.cg(my_mat, b, tol=tol, M=pre, maxiter=sz)
        return out, '{} sparse CG+P'.format('' if not succ else '*')
    time_method(sparsep)

    def minres():
        out, succ = scipy.sparse.linalg.minres(my_mat, b, tol=tol,
                                               maxiter=sz)
        return out, '{} sparse MINRES'.format('' if not succ else '*')
    time_method(minres)
    
    def minresp():
        out, succ = scipy.sparse.linalg.minres(my_mat, b, tol=tol,
                                               M=pre,
                                               maxiter=sz)
        return out, '{} sparse MINRES+P'.format('' if not succ else '*')
    time_method(minresp)

In [2]:
# prog n d q eps
sys.argv = ['', '1000', '4', '3', '1e-5']

print('* = no convergence')
utils.run_main(stress_sum_solve, '')

* = no convergence
size q 3 n 1000 d 4 eps 1e-05
random (well-cond) 
    cond 1664.9544339572756 tol 1e-06
            linear solve sec   0.7183 resid 1.8225e-05
               sparse CG sec   0.5590 resid 2.9578e-05
             sparse CG+P sec   0.5001 resid 8.5177e-06
           sparse MINRES sec   0.4949 resid 8.6360e-05
         sparse MINRES+P sec   0.3733 resid 1.9974e-03
linear decrease (poor-cond)
    cond 2738462.7709808256 tol 1e-06
            linear solve sec   0.7022 resid 3.6883e-05
               sparse CG sec  17.4549 resid 2.8188e-05
             sparse CG+P sec   0.1776 resid 3.4931e-05
           sparse MINRES sec  12.6250 resid 6.9144e-04
         sparse MINRES+P sec   0.1732 resid 4.6861e-05
exponentially decreasing (realistic)
    cond 41.97426199431347 tol 1e-06
            linear solve sec   0.7161 resid 4.3154e-05
               sparse CG sec   0.3760 resid 3.5697e-05
             sparse CG+P sec   0.2670 resid 9.5542e-06
           sparse MINRES sec   0.3017 

In [76]:
np.set_printoptions(precision=3)
ctr = 0
def i(_):
    global ctr
    ctr += 1

T = None
t = None
s = None
while True:
    t = np.random.randint(0, 100, 100)
    s = np.random.randint(0, 100, 100)
    t[::-1].sort()
    #t[0] = np.fabs(t[1:]).sum() + 1
    T = scipy.linalg.toeplitz(t)
    r = np.linalg.matrix_rank(T)
    print('rank', r)
    if r == 100:
        break

C = scipy.linalg.circulant(chan(t))
print('T^-1s')
b = (scipy.sparse.linalg.cg(T, s, tol=1e-10, maxiter=10000, callback=i)[0])
lcg = ctr
ctr =0
iC = np.linalg.inv(C)
lo = LinearOperator(C.shape, matvec=(lambda x: iC.dot(x)), rmatvec=(lambda x: iC.dot(x)))
ceig = fft(C[0])
def mv2(x):
    return ifft(fft(x) / ceig).real
lo2 = LinearOperator(C.shape, matvec=mv2, rmatvec=mv2)

c=(scipy.sparse.linalg.cg(T, s, tol=1e-10, M=lo, maxiter=10000, callback=i)[0])
pcg = ctr
ctr =0
cc=(scipy.sparse.linalg.cg(T, s, tol=1e-10, M=lo2, maxiter=10000, callback=i)[0])
pcg2 = ctr
ctr =0
print('LCG', lcg, 'PCG', pcg, 'PCG2', pcg2)
print('eq', np.linalg.norm(b- c), np.linalg.norm(b-cc))

rank 100
T^-1s
LCG 503 PCG 126 PCG2 159
eq 1.72147717396e-08 3.59656602828e-08


In [152]:
np.set_printoptions(precision=3)
ctr = 0
def i(_):
    global ctr
    ctr += 1

t = []
sz = 100
while len(t) < 2:
    tt = np.random.rand(sz)
    tt[::-1].sort()
    TT = scipy.linalg.toeplitz(tt)
    while True:
        r = np.linalg.matrix_rank(TT)
        if r == sz:
            break
        tt[0] *= 2
        TT = scipy.linalg.toeplitz(tt)

    t.append(tt)
        
s = np.random.randn(len(t) * len(t[0]))
Bs = np.array([[[3, 1], [1, 3]], [[2, 1], [1, 2]]])
tops = np.array(t)
tb = np.tensordot(Bs, tops, axes=(0, 0))

full = sum(np.kron(b, scipy.linalg.toeplitz(t)) for b, t in zip(Bs, t))
print(tb.shape)
circ_pre = np.array([[chan(tb[i, j])
    for j in range(tb.shape[1])] for i in range(tb.shape[0])])

C = np.bmat([[scipy.linalg.circulant(circ_pre[i, j])
    for j in range(tb.shape[1])] for i in range(tb.shape[0])]).A

b = (scipy.sparse.linalg.cg(full, s, tol=1e-10, maxiter=10000, callback=i)[0])
lcg = ctr
ctr =0

iC = np.linalg.inv(C)
lo = LinearOperator(C.shape, matvec=(lambda x: iC.dot(x)), rmatvec=(lambda x: iC.dot(x)))
ceig = fft(circ_pre)
rc = np.rollaxis(ceig, -1, 0)
lus = [scipy.linalg.lu_factor(dd, overwrite_a=True) for dd in rc]
# the rolled axis perm
perm=np.add.outer(np.arange(ceig.shape[2]), np.arange(ceig.shape[0]) * ceig.shape[2]).ravel()
inv_perm = np.zeros(len(perm), dtype=int)
inv_perm[perm] = np.arange(len(perm))

def mv2(x):
    return inv_precond(lus, perm, inv_perm, x).real
lo2 = LinearOperator(C.shape, matvec=mv2, rmatvec=mv2)

c=(scipy.sparse.linalg.cg(full, s, tol=1e-10, M=lo, maxiter=10000, callback=i)[0])
pcg = ctr
ctr =0
cc=(scipy.sparse.linalg.cg(full, s, tol=1e-10, M=lo2, maxiter=10000, callback=i)[0])
pcg2 = ctr
ctr =0
print('LCG', lcg, 'PCG', pcg, 'PCG2', pcg2)
print('eq', np.linalg.norm(b- c), np.linalg.norm(b-cc))

(2, 2, 100)
LCG 3324 PCG 922 PCG2 996
eq 6.21272915307e-08 1.07580824679e-07
